In [1]:
import collections
import os
import shutil
import tqdm

import numpy as np
import PIL.Image
import torch
import torchvision

In [2]:
torch.__version__

'1.5.1'

In [3]:
torch.version.cuda

'10.2'

In [4]:
torch.backends.cudnn.version()

7605

In [7]:
# torch.cuda.get_device_name(0)

In [9]:
torch.manual_seed(1)
# torch.cuda.manual_seed_all(2)

In [10]:
# CUDA_VISIBLE_DEVICES = 0,1 python train.py
# os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

In [11]:
torch.cuda.is_available()

False

In [12]:
# 计算图的随机性配置
torch.backends.cudnn.benchmark = True

In [13]:
# 计算图 确定性 配置
torch.backends.cudnn.deterministic = True

In [15]:
# torch.cuda.empty_cache()
# nvidia-smi --gpu-reset -i [gpu_id]

In [18]:
# tensor.type()
# tensor.size()
# tensor.dim()

# tensor = tensor.cuda()
# tensor = tensor.cpu()
# tensor = tensor.float()
# tensor = tensor.long()

In [ ]:
#  tensor -> np
ndarray = tensor.numpy()
# np -> tensor
tensor = torch.from_numpy(ndarray).float()
tensor = torch.from_numpy(ndarray.copy()).float()

In [ ]:
# tensor -> image
image = PIL.Image.fromarray(torch.clamp(tensor * 255, min=0, max=255).byte().permute(1, 2, 0).cpu().numpy())
# or
image = torchvision.transforms.functional.to_pil_image(tensor)
# image -> tensor
tensor = torch.from_numpy(np.asarray(PIL.Image.open(path))).permute(2, 0, 1).float() / 255
tensor = torchvision.transforms.functional.to_tensor(PIL.Image.open(path))


In [ ]:
# ndarray -> image
image = PIL.Image.fromarray(ndarray.astype(np.uint8))
# image -> ndarray
ndarray = np.asarray(PIL.Image.open(path))

In [ ]:
# tensor -> element
value = tensor.item()

In [ ]:
# reshape
tensor = torch.reshape(tensor, shape)
# shuffle 
# 1st dim
tensor = torch[torch.randperm(tensor.size(0))]
#  水平翻转 [ : :-1]
tensor = tensor[ : , : , : , torch.arange(tensor.size(3)-1, -1, -1).long()]

In [ ]:
# 3 different copy
# operation          New/Shared Memory.         still in computation graph
tensor.clone()          New                        Yes
tensor.detach()         Shared                     No
tensor.detach.clone()   New                        No

In [ ]:
# concact
tesnor = torch.cat(list_of_tensors, dim=0) # pd.DataFrame.concat
tensor = torch.stack(list_of_tensors, dim=0) # new dim

In [ ]:
# one-hot code
N = tensor.size(0)
one_hot = torch.zeros(N, num_classes).long()
one_hot.scatter(dim=1, index=torch.unsqueeze(tensor, dim=1), src=torch.ones(N, num_classes).long())


In [19]:
torch.nonzero(torch.Tensor([1,1,1,0,1]))

../torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


tensor([[0],
        [1],
        [2],
        [4]])

In [ ]:
# elements == 0
torch.nonezero(tensor) # indexs of nonzero
torch.nonzero(tensor == 0) # indexs of zero
torch.nonzero(tensor).size(0) # nums of nonzero
torch.nonzero(tensor==0).size(0) # nums of zero

In [23]:
# expend tensor 64 * 512 -> 64 * 512 * 7 * 7
tensor = torch.ones((64, 512))
torch.reshape(tensor, (64, 512, 1, 1)).expand(64, 512, 7, 7)

In [ ]:
# matrix multiplication
# (m*n ) * (n*p) -> (m*p)
res = torch.mm(tensor1, tensor2)
# batch mm
res = torch.bmm(tensor1, tensor2)
# element-wise
res = tensor1 * tensor2

In [6]:
x1 = torch.rand((3,2))
x1

tensor([[0.0591, 0.1834],
        [0.4650, 0.9579],
        [0.6641, 0.6408]])

In [7]:
x2 = torch.ones((2,2))
x2

tensor([[1., 1.],
        [1., 1.]])

In [12]:
x1[ : ,None, : ].shape == x1.unsqueeze(1).shape

True

In [13]:
# 计算x1 x2 两两之间的欧式距离: 利用广播机制
dist = torch.sqrt(
            torch.sum((x1.unsqueeze(1) - x2) ** 2,
            dim=2
    )
)
dist

tensor([[1.2459, 1.2459],
        [0.5366, 0.5366],
        [0.4918, 0.4918]])

In [ ]:
# define conv
conv = torch.nn.Conv2d(in_channels, out_channels,
                      kernel_size, stride, bias
                      )
# GAP（Global average pooling）层
gap = torch.nn.AdaptiveAvgPool2d(output_size=1)

In [ ]:
# 双线性汇合（bilinear pooling）
X = torch.reshape(N, D, H * W)                        # Assume X has shape N*D*H*W
X = torch.bmm(X, torch.transpose(X, 1, 2)) / (H * W)  # Bilinear pooling
assert X.size() == (N, D, D)
X = torch.reshape(X, (N, D * D))
X = torch.sign(X) * torch.sqrt(torch.abs(X) + 1e-5)   # Signed-sqrt normalization
X = torch.nn.functional.normalize(X)                  # L2 normalization

In [ ]:
# 多卡同步BN（Batch normalization）

# 当使用torch.nn.DataParallel将代码运行在多张GPU卡上时，
# PyTorch的BN层默认操作是各卡上数据独立地计算均值和标准差，
# 同步BN使用所有卡上的数据一起计算BN层的均值和标准差，
# 缓解了当批量大小（batch size）比较小时对均值和标准差估计不准的情况，
# 是在目标检测等任务中一个有效的提升性能的技巧。
sync_bn = torch.nn.SyncBatchNorm(num_features, eps=1e-05, momentum=0.1, affine=True, 
                                 track_running_stats=True)

In [ ]:
def convertBNtoSyncBN(module, process_group=None):
    '''Recursively replace all BN layers to SyncBN layer.

    Args:
        module[torch.nn.Module]. Network
    '''
    if isinstance(module, torch.nn.modules.batchnorm._BatchNorm):
        sync_bn = torch.nn.SyncBatchNorm(module.num_features, module.eps, module.momentum, 
                                         module.affine, module.track_running_stats, process_group)
        sync_bn.running_mean = module.running_mean
        sync_bn.running_var = module.running_var
        if module.affine:
            sync_bn.weight = module.weight.clone().detach()
            sync_bn.bias = module.bias.clone().detach()
        return sync_bn
    else:
        for name, child_module in module.named_children():
            setattr(module, name) = convert_syncbn_model(child_module, process_group=process_group))
        return module

In [ ]:
# 类似BN滑动平均
class BN(torch.nn.Module)
    def __init__(self):
        ...
        self.register_buffer('running_mean', torch.zeros(num_features))

    def forward(self, X):
        ...
        self.running_mean += momentum * (current - self.running_mean)

In [ ]:
# 统计 参数量
num_parameters = sum(
        torch.numel(parameter) for parameter in model.parameters()
    )

In [24]:
# summary 遍历 model layers 间 矩阵大小的变化
from torchsummary import summary # pip install torchsummary
summary(your_model, input_size(channels, H, W))

In [25]:
import torch.nn as nn
import torch.nn.functional as F

In [26]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    


In [27]:
model = Net()
summary(model, (1, 28, 28)) # as kernerl_size = 5 默认步长 stride=1

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 24, 24]             260
            Conv2d-2             [-1, 20, 8, 8]           5,020
         Dropout2d-3             [-1, 20, 8, 8]               0
            Linear-4                   [-1, 50]          16,050
            Linear-5                   [-1, 10]             510
Total params: 21,840
Trainable params: 21,840
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 0.08
Estimated Total Size (MB): 0.15
----------------------------------------------------------------


In [ ]:
# common practice for initialization
for layer in model.modulers():
    if isinstance(layer, torch.nn.Conv2d):
        torch.nn.init.kaiming_normal_(layer.weight, model='fan_out',
                                     nonlinearity='relu')
        if layer.bias is not None:
            torch.nn.init.constant_(layer.bias, val=0.0)
    elif isinstance(layer, torch.nn.BatchNorm2d):
        torch.nn.init.constant_(layer.weight, val=1.0)
        torch.nn.init.constant_(layer.bias, val=0.0)
    elif isinstance(layer, torch.nn.Linear):
        torch.nn.init.xavier_normal_(layer.weight)
        if layer.bias is not None:
            torch.nn.init.constant_(layer.bias, val=0.0)
            
# initialization with given tensor
layer.weight = torch.nn.Parameter(tensor)

In [ ]:
# load pre-train model
model.load_state_dict(torch.load('model-path'), strict=False)
# cpu
model.load_state_dict(torch.load('model-path'), map_location='cpu')

In [ ]:
# fine-tuning 
model = torchvision.models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
model.fc = nn.Linear(512, 100)
optimizer = torch.optim.SGD(model.fc.parameters(), lr=1e-2, 
                            momentum=0.9, weight_decay=1e-4
                           )

In [ ]:
# fine-tuning with different lr
parameters = [{'params': conv_parameters, 'lr': 1e-3}, 
              {'params': model.fc.parameters()}]
optimizer = torch.optim.SGD(parameters, lr=1e-2, 
                            momentum=0.9, weight_decay=1e-4)

In [ ]:
# training
for t in epoch(80):
    for images, labels in tqdm.tqdm(train_loader, desc='Epoch %3d' % (t + 1)):
#         images, labels = images.cuda(), labels.cuda()
        scores = model(images)
        loss = loss_function(scores, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
# 标记平滑（label smoothing
# 略

In [ ]:
# Mixup
# 略

In [ ]:
# L1 Reg
l1_regularization = torch.nn.L1Loss(reduction='sum')
# or 
# loss = nn.CrossEntropyLoss()
for param in model.parameters():
    loss += lambda_ * torch.sum(torch.abs(param))

loss.backward()

In [ ]:
# 针对某些参数不参与 weight decay
bias_list = (param for name, param in model.named_parameters() 
            if name[-4: ] == 'bias')
others_list = (param for name, param in model.named_parameters() 
            if name[-4: ] != 'bias')
parameters = [{'parameters': bias_list, 'weight_decay': 0},
             {'parameters': others_list}
             ]
optimizer = torch.optim.SGD(paramters, lr=1e-2, momentum=0.9, weight_decay=1e-4)

In [ ]:
#梯度裁剪（gradient clipping）
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=20)

In [ ]:
# output
score = model(images)
prediction = torch.argmax(score, dim=1)
num_correct = torch.sum(prediction == labels).item()
acc = num_correct / labels.size(0)

In [ ]:
# 模型可视化
# visdom
# or
# tensorboard

In [ ]:
# 获取训练过程中的learning rate
# global lr
lr = next(iter(optimizer.param_groups))['lr']
# mutilple lr
all_lr = []
for param_group in optimizer.param_groups:
    all_lr.append(param_group['lr'])

In [ ]:
# 调整lr
scheluder = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max')

for t in range(0,90):
    train...
    scheduler.step(val_acc)

In [ ]:
# 保存模型断点: stat_dict
is_best = current_acc > best_acc
best_acc = max(best_acc, current_acc)
checkpoint = {
    'best_acc':best_acc,
    'epoch': t+1,
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
}
model_path = os.path.join('model', 'checkpoint.pt.tar')
torch.save(checkpoint, 'checkpoin.path.tar')
if is_best:
    shutil.copy('checkpoin.pt.tar', model_path)

# load
if resume:
    model_path = os.path.join('model', 'checkpoint.pt.tar')
    assert os.path.isfile(model_path)
    checkpoint = torch.load(mdoel_path)
    best_acc = checkpoint['best_acc']
    start_epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['model'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    print('Load Checkpoin at Epoch %d' % (start_epoch))

In [ ]:
# 保存模型: pickle
torch.save(model, PATH)
# load
model = torch.load(PATH)

In [29]:
def log_softmax(x):
    return x - x.exp().sum(-1).log().unsqueeze(-1)

In [30]:
x1

tensor([[0.0591, 0.1834],
        [0.4650, 0.9579],
        [0.6641, 0.6408]])

In [34]:
log_softmax(x1)

tensor([[-0.7572, -0.6329],
        [-0.9697, -0.4768],
        [-0.6816, -0.7049]])

In [35]:
x1.exp()

tensor([[1.0609, 1.2013],
        [1.5921, 2.6063],
        [1.9427, 1.8980]])

In [36]:
x1.exp().sum(-1)

tensor([2.2621, 4.1983, 3.8407])

In [37]:
x1.exp().sum(-1).log()

tensor([0.8163, 1.4347, 1.3457])

In [38]:
x1.exp().sum(-1).log().unsqueeze(-1)

tensor([[0.8163],
        [1.4347],
        [1.3457]])

In [39]:
torch.nn.CrossEntropyLoss?